## 음성인식 : Feature representation (Speech_Recognition : Feature representation)
---
__참고사항 :__ 
이 글은 성원용 교수님의 "음성인식 및 합성" 강의와 [Deep Learning for Speech Recognition(Adam Coates, Baidu)](https://www.youtube.com/watch?v=g-sndkf7mCs)를 주로 참고하여 정리하였습니다.  
정리글이므로 오류가 있을 수 있으니 github issue 란에 제보해주시면 확인 후 수정하겠습니다.  
감사합니다.
<br \>

### 5. MFCC (Mel Frequency Cepstral Coefficient)
![empty](./img/MFCC.png "MFCC")  
출처 : cs224s slide  
<br \>
음성 wave에서 pitch 정보를 없애기 위한 전처리 과정이다. 그러나 아무리 필기를 다시 봐도 MFCC를 정리하기에는 이해도가 부족하여,  
Mel scaled filter bank까지는 필기로 정리하고, 나머지는 인터넷 검색을 토대로 정리할 것이다.  

주로 도움 받은 글은 다음과 같다.  
[Mel Frequency Cepstral Coefficient (MFCC) 란 무엇인가? - 음성 인식 알고리즘](https://m.blog.naver.com/PostView.nhn?blogId=mylogic&logNo=220988857132&proxyReferer=https%3A%2F%2Fwww.google.co.kr%2F)  
[Mel Frequency Cepstral Coefficient (MFCC) tutorial](http://practicalcryptography.com/miscellaneous/machine-learning/guide-mel-frequency-cepstral-coefficients-mfccs/)  
[MFCC feature를 얻는 코드](https://translate.googleusercontent.com/translate_c?depth=1&hl=ko&prev=search&rurl=translate.google.co.kr&sl=en&sp=nmt4&u=https://github.com/jameslyons/python_speech_features&xid=17259,15700022,15700105,15700124,15700149,15700168,15700173,15700186,15700201&usg=ALkJrhh-YreSnLrLA7J6xThIwrA2lnIb0w)  
[cs224s feature extraction PDF](http://web.stanford.edu/class/cs224s/lectures/224s.17.lec5.pdf)  
좋은 글 감사합니다(\_ \_).  
<br \>
- __Pre-emphasis__  
음성 wave는 낮은 주파수는 잘 얻어지지만, 상대적으로 높은 주파수는 잘 안 얻어지는 특징이 있다.  
그래서 높은 주파수를 증폭시키는 것을 Pre-emphasis라고 한다. 수식은 아래와 같다.  
<br \>
$$ y[n] = x[n] - \alpha x[n-1] $$
![empty](./img/preemphasis.png "preemphasis")
극단적으로 주파수가 아주 낮은 왼쪽 그림에서는 Pre-emphasis 결과값들이 0이지만,  
상대적으로 주파수가 높은 오른쪽 그림에서는 Pre-emphasis 결과값들이 이전보다 더 크게 진동하게 된다는 것을 알 수 있다.  
이는 주파수가 더 높으면 높을록 더 크게 진동될 것이다.  
<br \>
- __Windowing__   
높은 주파수가 증폭된 음성 wave는 windowing을 통해 부분 부분 DFT(Discret Fourier Transform)을 거쳐 주파수 영역 표현(Frequency domain representation)으로 바뀌게 된다(실제로는 DFT의 빠른 버젼인 [고속 푸리에 변환(Fast Fouriere Transform)](https://ko.wikipedia.org/wiki/%EA%B3%A0%EC%86%8D_%ED%91%B8%EB%A6%AC%EC%97%90_%EB%B3%80%ED%99%98)을 사용한다).  
<br \>
여기서 windowing을 쓰는 이유는 너무 긴 신호에서는 고속 푸리에 변환이 작동하지 않기 때문이라고 한다.  
참고 : [Why should one use windowing functions for FFT?](https://dsp.stackexchange.com/questions/11312/why-should-one-use-windowing-functions-for-fft)  
<br \>
실습 시에는 25msec 크기의 window를 10msec 간격으로 sliding 해서 진행하였다.  
![empty](./img/windowing.png "windowing")
출처 : [Deep Learning for Speech Recognition(Adam Coates, Baidu)](https://www.youtube.com/watch?v=g-sndkf7mCs)  
<br \>
__Hamming window__  
Windowing을 할 때 많은 경우 Hamming window 기법을 사용하는데, 그 이유는 Hamming Window가 window 간 경계선 부분을 흐려주고 중심만 강조시켜주는 역할을 하기 때문이다. 
<br \>                             
$$ \mbox{hamming      } w[n] = \begin{cases}
                               0.54 - 0.46\mbox{ cos}(\frac{2\pi n}{L}) & \mbox{}0\le n \le L-1 \\ 
                               0 & \mbox{otherwise} 
                               \end{cases}
                               $$  
<br \>
Hamming window을 적용하면 아래와 같다. 끝부분이 줄어든 형태로 나타난다.  
![empty](./img/hamming_window.png "hamming_window")
출처 : [dsprelated.com](https://www.dsprelated.com/freebooks/sasp/Resolving_Sinusoids.html)   
<br \>  
- __DFT__  
위 spectrogram 그림처럼 Hammed window를 가져와 빠른 DFT 버전인 FFT에 적용시킨다.  
이때 FFT의 결과값에 절대값을 씌우고 제곱을 하여 에너지를 구해준다.  
<br \>  
- __Mel scaled fitler bank__  
사람의 청각은 높은 주파수에서 덜 민감하다. 즉 낮은 주파수의 변화를 더 잘 알아차린다.  
따라서 음성 인식을 모델링 할 때 사람 청각의 이런 특징을 반영해주어야 하는데, 이때 사용하는 것이 Mel scaled filter bank이다.  
<br \>  
$$ Mel(f) = 1125 \ln{\left(1 + \frac{f}{700}\right)} $$
참고 : [Mel Frequency Cepstral Coefficient (MFCC) 란 무엇인가? - 음성 인식 알고리즘](https://m.blog.naver.com/PostView.nhn?blogId=mylogic&logNo=220988857132&proxyReferer=https%3A%2F%2Fwww.google.co.kr%2F)    
![empty](./img/mel_scaled_filter_bank.png "mel_scaled_filter_bank")
출처 : [Detecting Patients with Parkinson's disease using Mel Frequency Cepstral Coefficients and Support Vector Machines](https://www.researchgate.net/publication/280027126_Detecting_Patients_with_Parkinson's_disease_using_Mel_Frequency_Cepstral_Coefficients_and_Support_Vector_Machines)   
<br \>
낮은 주파수 정보는 촘촘하게 하여 정보량을 늘리고, 높은 주파수 정보는 널널하게 하여 정보량을 줄이고 있다.  
통상 저 삼각형의 개수는 26개 정도를 쓴다고 한다.  
<br \>
Mel scaled filter bank는 아래 그림과 같이 적용한다.  
FFT 결과값으로 구한 에너지와 Filter bank를 행렬곱하면 된다.  
그리고 log를 씌워준다. log를 취하면 인간이 실제로 듣는 것과 유사한 소리의 특징을 만들 수 있다고 한다.  
(인간의 귀는 소리의 크기를 Linear scale로 감지하는 것이 아니기 때문)  
참고 : [Mel Frequency Cepstral Coefficient (MFCC) 란 무엇인가? - 음성 인식 알고리즘](https://m.blog.naver.com/PostView.nhn?blogId=mylogic&logNo=220988857132&proxyReferer=https%3A%2F%2Fwww.google.co.kr%2F)  
![empty](./img/get_mel_scaled_filter_bank.png "get mel scaled filter bank")
출처 : cs224s slide  
<br \>
- __이산 코사인 변환(Discret Cosine Transform, DCT)__  
Filter bank는 모두 Overlapping 되어 있기 때문에 Filter bank 에너지들 사이에 상관관계가 존재하게 되는데,  
DCT를 사용해 에너지들의 상관관계를 분리하게 된다.  
1-d DCT 식은 다음과 같다.
$$ X_c(k) = (1/N)\sum_{n=0}^{N-1} X_n \cos (k2\pi n/N), \mbox{where k=0,,2,...,N-1} $$
참고 : [XILINX, one-dimensional DCT Equation](http://homepages.cae.wisc.edu/~ece554/website/Xilinx/app_notes/DCT_IDCT%20Customer%20Tutorial%20custdct.pdf)  
<br \>
- __Keep DCT Coefficient__   
DCT Coefficient는 Filter bank energy 수 만큼 나온다. 통상 26개가 나오게 될 것이다.  
여기서 DCT Coefficient가 낮은 순으로 12개만 남기고 버린다. 그 이유가 바로 pitch를 제거하기 위한 것이다.  
DCT Coefficient가 높을수록 Filter bank enery의 빠른 변화를 나타낸다.  
즉, DCT Coefficient가 높은 것을 제거한다는 뜻은 주파수가 급변하는 부분을 제거하는 효과가 생기고  
이것이 MFCC 이후 남성과 여성의 음성이 중성적으로 바뀌게 되는 요인이 되는 것이다.  
참고 : [Mel Frequency Cepstral Coefficient (MFCC) tutorial](http://practicalcryptography.com/miscellaneous/machine-learning/guide-mel-frequency-cepstral-coefficients-mfccs/)  
<br \>
- __Delta and Delta-Deltas__  
DCT까지 거쳐서 구한 MFCC feature vector는 single frame만 고려한 것이다.  
음성은 dynamic하기 때문에 이를 고려해주면 좋은데, 그 방법이 Delta and Delta-Deltas이다. 이 방법을 적용하면 ASR 성능이 좋아진다고 한다.
<br \>
dela coefficient를 구하는 방법은 다음과 같다.  
<br \>
$$ d_t = \frac {\sum_{n=1}^{N} n(C_{t+n} - c_{t-n})} {2 \sum_{n=1}^{N} n^2} $$ 
<br \>
참고 : [Mel Frequency Cepstral Coefficient (MFCC) tutorial](http://practicalcryptography.com/miscellaneous/machine-learning/guide-mel-frequency-cepstral-coefficients-mfccs/)  
<br \>
**(의문)이 장 맨 위에 첨부한 cs224의 MFCC 그림을 보면 derivatives(이게 Delta and Delta-Deltas 부분인 듯) 전에 IFFT 부분이 들어간다.  
즉, DCT Coefficient 12개 뽑고 다시 time domain으로 바꿔준다는 것인데, MFCC tutorial site에 보면 IFFT에 대한 언급이 없다.  
이 부분이 의문**

Acoustic model을 DNN을 쓰거나 아예 Acoustic model과 Language model을 RNN 기반 모델을 사용하는 경우, 보통 Mel scaled filter bank까지 적용한 데이터를 input으로 사용한다.  
하지만 Acoustic model으로 GMM을 사용하는 경우에는 MFCC feature vector를 사용한다. 